# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from nltk.stem import SnowballStemmer
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

df = pd.read_csv('/datasets/data.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


children — количество детей в семье
days_employed — общий трудовой стаж в днях
dob_years — возраст клиента в годах
education — уровень образования клиента
education_id — идентификатор уровня образования
family_status — семейное положение
family_status_id — идентификатор семейного положения
gender — пол клиента
income_type — тип занятости
debt — имел ли задолженность по возврату кредитов
total_income — ежемесячный доход
purpose — цель получения кредита

**Вывод**: При первом осмотре данные собраны коректно, за исключением данных по трудовому стажу, отрицательные значения, и данные указаны явно не в днях. Так же есть пропуски в стаже и зарплате, нужно разобраться от чего появились эти пропуски. Цели кредита можно сгруппировать по категориям, а колонку образование привести к нижнему регистру

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
display(df.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для начала ознакомимся с пропущеными значениями выведим их в отдельную таблицу.

In [5]:
empty_df = df.loc[df['days_employed'].isna() == True] #Создаем таблицу с пропущенными значениями.
display(empty_df.head(10))
display(empty_df.count()) 
display(empty_df['income_type'].value_counts()) #Подсчитаем количество значений, по категориям занятости

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Проведя проверку данных, пришли к выводу, что даныне пропущенные в трудовом стаже также и пропущены в столбце зарплаты. Из этого можно предположить что данные отсутсвуют из-за информации о работе. Сходство по типу занятости не обноружены.

In [6]:
# Выясним какие строки в графе стажа принимают положительные значения, и выясним уникальные значени в графе занятости
days_employed_mz = df[df['days_employed'] > 0]
display(days_employed_mz.head(5))
display(days_employed_mz['income_type'].unique())
display(days_employed_mz['income_type'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


array(['пенсионер', 'безработный'], dtype=object)

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [7]:
def employed_period_possible(row):
    years_employed = row['years_employed']
    years_employed_realistic = row['years_employed_realistic']
    if years_employed < 0:
        years_employed = years_employed * (-1)
    if years_employed_realistic > years_employed:
        return 'possible'
    else:
        return 'impossible'

df['years_employed'] = df['days_employed'] / 365.25
df['years_employed_realistic'] = df['dob_years'] - 18
df['years_employed_possibility'] = df.apply(employed_period_possible, axis = 1)
display(df.head(5))
display(df['years_employed_possibility'].value_counts())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,years_employed_realistic,years_employed_possibility
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,-23.101090,24,possible
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,-11.019312,18,possible
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,-15.396092,15,impossible
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,-11.292942,14,possible
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,931.597733,35,impossible


possible      15427
impossible     6098
Name: years_employed_possibility, dtype: int64

После перевода трудового стажа в года, сравнили с возрастом заемщиков, и вывели возможно или не возможно иметь такой трудовой стаж при условии что в стране разрешенно работать с 18 лет. К правильности собранных данных в колонке 'days_employed' много вопросов, т.к. эта колонка не влияет на поставленую задачу, оставляем ее как есть

In [8]:
data_grouped_income = df.groupby('income_type')['total_income'].median()
display(data_grouped_income)
income_type_dict = data_grouped_income.to_dict()

def get_median_income_by_income_type(row):
    income_type = row['income_type']
    return int(income_type_dict[income_type])

df['median_income'] = df.apply(get_median_income_by_income_type, axis = 1)
df['total_income'] = df['total_income'].fillna(df['median_income'])
df.info()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 16 columns):
children                      21525 non-null int64
days_employed                 19351 non-null float64
dob_years                     21525 non-null int64
education                     21525 non-null object
education_id                  21525 non-null int64
family_status                 21525 non-null object
family_status_id              21525 non-null int64
gender                        21525 non-null object
income_type                   21525 non-null object
debt                          21525 non-null int64
total_income                  21525 non-null float64
purpose                       21525 non-null object
years_employed                19351 non-null float64
years_employed_realistic      21525 non-null int64
years_employed_possibility    21525 non-null object
median_income                 21525 non-null int64
dtypes: float64(3), int64(7), object(6)
memory usage: 2.6+ MB


Заполнить пропущенные значени в total_income было решено медиаными значениями по категориям собранными из столбца income_type.

**Вывод**  Было выявлено 2174 пропуска в столбцах days_employed и total_income так же пропуски в этих столбцах совпадают, а сходство по income_type не выявлено. Будем считать что эти пропуски полностью случайны. Столбец days_empolyed оставим без изменений т.к. он не влияет на постовленную задачу. Столбец total_income заполнили медианными значениями по категориям из income_type.

### Замена типа данных

In [9]:
#Преведем вещественные значения к целочисленным
df['days_employed'] = df['days_employed'].fillna(0)
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')
df['years_employed'] = df['years_employed'].fillna(0)
df['years_employed'] = df['years_employed'].astype('int64')
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 16 columns):
children                      21525 non-null int64
days_employed                 21525 non-null int64
dob_years                     21525 non-null int64
education                     21525 non-null object
education_id                  21525 non-null int64
family_status                 21525 non-null object
family_status_id              21525 non-null int64
gender                        21525 non-null object
income_type                   21525 non-null object
debt                          21525 non-null int64
total_income                  21525 non-null int64
purpose                       21525 non-null object
years_employed                21525 non-null int64
years_employed_realistic      21525 non-null int64
years_employed_possibility    21525 non-null object
median_income                 21525 non-null int64
dtypes: int64(10), object(6)
memory usage: 2.6+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,years_employed_realistic,years_employed_possibility,median_income
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-23,24,possible,142594
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-11,18,possible,142594
2,0,-5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-15,15,impossible,142594
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-11,14,possible,142594
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,931,35,impossible,118514


### Обработка дубликатов

In [10]:
# Проверим на явные дубликаты
print(df['children'].unique())
print(df['education'].unique())
print(df['family_status'].unique()) 
print(df['income_type'].unique())
print(df['gender'].unique())

[ 1  0  3  2 -1  4 20  5]
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['F' 'M' 'XNA']


In [11]:
#В столбце дети "-1" и "20" выглядят странно, скорее всего это опечатка или техническая ошибка, заменим эти значения на более реальные.
df['children'] = df['children'].replace(-1, 1)
df['children'] = df['children'].replace(20, 2)

In [12]:
# Столбец education приведем к нижему регистру
df['education'] = df['education'].str.lower()

# Проверим на правильность исполнение команды
print(df['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


In [13]:
df_duplicated = df[df.duplicated() == True]
df_duplicated.sort_values('dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,years_employed_realistic,years_employed_possibility,median_income
20297,1,0,23,среднее,1,гражданский брак,1,F,сотрудник,0,142594,сыграть свадьбу,0,5,impossible,142594
19321,0,0,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,142594,сделка с подержанным автомобилем,0,5,impossible,142594
18328,0,0,29,высшее,0,женат / замужем,0,M,сотрудник,0,142594,покупка жилой недвижимости,0,11,impossible,142594
21281,1,0,30,высшее,0,женат / замужем,0,F,сотрудник,0,142594,покупка коммерческой недвижимости,0,12,impossible,142594
6312,0,0,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости,0,12,impossible,142594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17338,0,0,64,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,0,46,impossible,118514
20187,0,0,65,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу,0,47,impossible,118514
9528,0,0,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,118514,операции со своей недвижимостью,0,48,impossible,118514
9604,0,0,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы,0,53,impossible,118514


In [14]:
display(df.shape)
display(df.duplicated().sum())
df = df.drop_duplicates().reset_index(drop = True)
display(df.shape)

(21525, 16)

71

(21454, 16)

**Вывод** В столбце education все строки были приведены к единому регистру, и проверены еще раз на явные дубли. После с помощью метода duplicated. Было выявлено 71 дубликат. Все дубли были удалены. Возможная причина, повторная заявка на кредит.

### Лемматизация

In [15]:
credit_pur = df['purpose'].unique() # Проведем визуальный анализ на схожесть категорий для дальнейшей лемматизации

In [16]:
string = '; '.join(credit_pur)
print(string)

покупка жилья; приобретение автомобиля; дополнительное образование; сыграть свадьбу; операции с жильем; образование; на проведение свадьбы; покупка жилья для семьи; покупка недвижимости; покупка коммерческой недвижимости; покупка жилой недвижимости; строительство собственной недвижимости; недвижимость; строительство недвижимости; на покупку подержанного автомобиля; на покупку своего автомобиля; операции с коммерческой недвижимостью; строительство жилой недвижимости; жилье; операции со своей недвижимостью; автомобили; заняться образованием; сделка с подержанным автомобилем; получение образования; автомобиль; свадьба; получение дополнительного образования; покупка своего жилья; операции с недвижимостью; получение высшего образования; свой автомобиль; сделка с автомобилем; профильное образование; высшее образование; покупка жилья для сдачи; на покупку автомобиля; ремонт жилью; заняться высшим образованием


In [17]:
lemmas = m.lemmatize(string)
print(Counter(lemmas))

Counter({' ': 59, '; ': 37, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


In [18]:
russian_stemmer = SnowballStemmer('russian')

In [19]:
#Напишем функцию для лемматизации явных вариантов
#goals = ['недвижимость', 'жилой', 'авто', 'образование', 'свадьба']

def group_by_stem(row):
    purpose_group = m.lemmatize(row['purpose'])
    if 'недвижимость' in purpose_group:
        return 'недвижимость'
    elif 'жилье' in purpose_group:
        return 'недвижимость'
    elif 'автомобиль' in purpose_group:
        return 'автомобиль'
    elif 'образование' in purpose_group:
        return 'образование'
    elif 'свадьба' in purpose_group:
        return 'свадьба'

In [20]:
df['purpose_grouped'] = df.apply(group_by_stem, axis = 1)
display(df['purpose_grouped'].unique())
display(df.head(10))


array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,years_employed_realistic,years_employed_possibility,median_income,purpose_grouped
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-23,24,possible,142594,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-11,18,possible,142594,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-15,15,impossible,142594,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-11,14,possible,142594,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,931,35,impossible,118514,свадьба
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,-2,9,possible,172357,недвижимость
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,-7,25,possible,172357,недвижимость
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,0,32,possible,142594,образование
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,-18,17,impossible,142594,свадьба
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,-5,23,possible,142594,недвижимость


**Вывод** После лемматизации можно будеть выделить группы для предоставления данных.

### Категоризация данных

In [21]:
df_debt_all = df.groupby('debt')['debt'].agg('count') #Подсичтаем кол-во задолжностей
df_debt_all_perc = (df_debt_all[1] / df_debt_all.sum()) * 100 #Узнаем процент задолжностей по отношению к тем у кого их нет 
display(df_debt_all)
display(df_debt_all_perc)

debt
0    19713
1     1741
Name: debt, dtype: int64

8.115036822970076

In [22]:
def group_by_children_quantity(row): # Функция для группировки по кол-ву детей.
    children = row['children']
    if children <= 0:
        return '0'
    elif children == 1:
        return '1'
    else:
        return '2+'

df['children_grouped'] = df.apply(group_by_children_quantity, axis = 1)
display(df.head(5))
grouped_by_children = df.groupby('children_grouped')['debt'].agg(['count', 'sum'])
grouped_by_children['%'] = (grouped_by_children['sum'] / grouped_by_children['count']) * 100
display(grouped_by_children.sort_values('%'))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,years_employed_realistic,years_employed_possibility,median_income,purpose_grouped,children_grouped
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,-23,24,possible,142594,недвижимость,1
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,-11,18,possible,142594,автомобиль,1
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,-15,15,impossible,142594,недвижимость,0
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,-11,14,possible,142594,образование,2+
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,931,35,impossible,118514,свадьба,0


,count,sum,%
children_grouped,,,
0,14091,1063,7.543822
1,4855,445,9.165808
2+,2508,233,9.290271


In [23]:
#Группируем по семейному положению
grouped_by_family_status = df.groupby('family_status')['debt'].agg(['count', 'sum'])
grouped_by_family_status['%'] = (grouped_by_family_status['sum'] / grouped_by_family_status['count']) * 100
display(grouped_by_family_status.sort_values('%'))

,count,sum,%
family_status,,,
вдовец / вдова,959,63,6.569343
в разводе,1195,85,7.112971
женат / замужем,12339,931,7.545182
гражданский брак,4151,388,9.347145
Не женат / не замужем,2810,274,9.750890


In [24]:
#Группируем по целям кредита
grouped_by_purpose = df.groupby('purpose_grouped')['debt'].agg(['count', 'sum'])
grouped_by_purpose['%'] = (grouped_by_purpose['sum'] / grouped_by_purpose['count']) * 100
display(grouped_by_purpose.sort_values('%'))

,count,sum,%
purpose_grouped,,,
недвижимость,10811,782,7.233373
свадьба,2324,186,8.003442
образование,4013,370,9.220035
автомобиль,4306,403,9.359034


In [25]:
grouped_by_gender = df.groupby('gender')['debt'].agg(['count', 'sum'])
grouped_by_gender['%'] = (grouped_by_gender['sum'] / grouped_by_gender['count']) * 100
display(grouped_by_gender.sort_values('%'))

,count,sum,%
gender,,,
XNA,1,0,0.000000
F,14174,994,7.012840
M,7279,747,10.262399


In [26]:
# Сгруппируем по образованию
grouped_by_education = df.groupby('education')['debt'].agg(['count', 'sum'])
grouped_by_education['%'] = (grouped_by_education['sum'] / grouped_by_education['count']) * 100
display(grouped_by_education.sort_values('%'))

,count,sum,%
education,,,
ученая степень,6,0,0.000000
высшее,5250,278,5.295238
среднее,15172,1364,8.990245
неоконченное высшее,744,68,9.139785
начальное,282,31,10.992908


In [27]:
# Напишем функию для объедения возраста в категории
def group_by_age(row):
    age = row['dob_years']
    if age <= 30:
        return '18-30'
    elif 30 < age <= 40:
        return '30-40'
    elif 40 < age <= 45:
        return '40-45'
    elif 45 < age <= 50:
        return '45-50'
    elif 50 < age <= 55:
        return '50-55'
    else:
        return '55+'

df['age_grouped'] = df.apply(group_by_age, axis = 1)
#display(data.head(5))
#Сгруппируем по собраным возрастным категориям
grouped_by_age = df.groupby('age_grouped')['debt'].agg(['count', 'sum'])
grouped_by_age['%'] = (grouped_by_age['sum'] / grouped_by_age['count']) * 100
display(grouped_by_age.sort_values('%'))

,count,sum,%
age_grouped,,,
55+,4336,240,5.535055
50-55,2308,141,6.109185
45-50,2506,187,7.462091
40-45,2754,216,7.843137
30-40,5732,546,9.525471
18-30,3818,411,10.764798


In [28]:
#Сделаем группировку по типу занятости
grouped_by_income_type = df.groupby('income_type')['debt'].agg(['count', 'sum'])
grouped_by_income_type['%'] = (grouped_by_income_type['sum'] / grouped_by_income_type['count']) * 100
display(grouped_by_income_type.sort_values('%'))

,count,sum,%
income_type,,,
предприниматель,2,0,0.000000
студент,1,0,0.000000
пенсионер,3829,216,5.641160
госслужащий,1457,86,5.902539
компаньон,5078,376,7.404490
сотрудник,11084,1061,9.572357
безработный,2,1,50.000000
в декрете,1,1,100.000000


In [29]:
# Распределим уровень дохода на 4 группы, присвоим статусы каждой группе
def income_status(monthly_income):
    if monthly_income <= 90000.0:
            return 'Низкий уровень дохода'
    if monthly_income <= 135000.0:
            return 'Средний уровень дохода'
    if monthly_income < 195000.0:
            return 'Высокий уровень дохода'
    return 'Сверхвысокий уровень дохода'
df['income_status'] = df['total_income'].apply(income_status)

In [30]:
#Соберем сводную таблицу
df_pivot = df.pivot_table(index = ['family_status', 'gender', 'education'], columns = 'debt', values = 'purpose', aggfunc = 'count')
df_pivot['%'] = (df_pivot[1] / df_pivot[0]) * 100
df_pivot.sort_values('%')

debt                                                   0      1          %
family_status         gender education                                    
Не женат / не замужем F      высшее                503.0   19.0   3.777336
вдовец / вдова        F      высшее                119.0    5.0   4.201681
в разводе             F      высшее                242.0   11.0   4.545455
женат / замужем       F      высшее               1858.0   97.0   5.220667
гражданский брак      F      высшее                650.0   35.0   5.384615
женат / замужем       M      высшее               1027.0   58.0   5.647517
вдовец / вдова        F      среднее               695.0   45.0   6.474820
в разводе             M      высшее                 59.0    4.0   6.779661
                      F      среднее               601.0   45.0   7.487521
гражданский брак      M      неоконченное высшее    53.0    4.0   7.547170
женат / замужем       F      среднее              5111.0  409.0   8.002348
гражданский брак      M      высшее                248.0   20.0   8.064516
Не женат / не замужем F      начальное              12.0    1.0   8.333333
                             неоконченное высшее    94.0    8.0   8.510638
                             среднее              1002.0   90.0   8.982036
женат / замужем       F      неоконченное высшее   197.0   18.0   9.137056
                             начальное              62.0    6.0   9.677419
гражданский брак      F      среднее              1831.0  181.0   9.885309
                             неоконченное высшее    98.0   10.0  10.204082
Не женат / не замужем M      начальное              19.0    2.0  10.526316
женат / замужем       M      неоконченное высшее   114.0   12.0  10.526316
Не женат / не замужем M      высшее                256.0   27.0  10.546875
женат / замужем       M      среднее              2971.0  322.0  10.838102
в разводе             M      среднее               162.0   19.0  11.728395
Не женат / не замужем M      неоконченное высшее    81.0   10.0  12.345679
в разводе             M      неоконченное высшее     8.0    1.0  12.500000
женат / замужем       M      начальное              64.0    9.0  14.062500
гражданский брак      M      среднее               837.0  127.0  15.173238
в разводе             F      начальное              13.0    2.0  15.384615
                             неоконченное высшее    19.0    3.0  15.789474
вдовец / вдова        F      неоконченное высшее    11.0    2.0  18.181818
                      M      высшее                 10.0    2.0  20.000000
Не женат / не замужем M      среднее               568.0  117.0  20.598592
гражданский брак      F      начальное              33.0    7.0  21.212121
вдовец / вдова        M      среднее                30.0    9.0  30.000000
гражданский брак      M      начальное              12.0    4.0  33.333333
Не женат / не замужем M      ученая степень          1.0    NaN        NaN
в разводе             M      начальное               5.0    NaN        NaN
                             ученая степень          1.0    NaN        NaN
вдовец / вдова        F      начальное              27.0    NaN        NaN
                      M      начальное               4.0    NaN        NaN
гражданский брак      XNA    неоконченное высшее     1.0    NaN        NaN
женат / замужем       F      ученая степень          2.0    NaN        NaN
                      M      ученая степень          2.0    NaN        NaN

**Вывод** 
children - по кол-ву детей было много разных данных, поэтому для удобства анализа собрал все данные в 3 группы: нет детей, один ребенок и 2+ детей. А также заменили артефакты "-1" и "20"

family_status - вариантов не так много, поэтому не стал разбивать на группы.

total_income - разбивал на группы сначала интуитивно, а затем поправил группы для того, чтобы вес у них был более менее одинаковый.

purpose - после применения .unique() выявил 4 основных направления: недвижимость, свадьба, образование и автомобиль, по которым и произвел группировку.

gender - вариантов не так много, поэтому не стал разбивать на группы. А так же есть строка с неопределенным полом. Скорее всего поле было не заполнено, оставим как есть, процент влияение одной строки крайне мал

education - вариантов не так много, поэтому не стал разбивать на группы.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
children_pivot = df.pivot_table(index = ['children_grouped'], \
                                columns = ['debt'], \
                                values = 'purpose', aggfunc='count')

children_pivot['%'] = children_pivot[1] / (children_pivot[1] + children_pivot[0])  * 100
children_pivot

debt,0,1,%
children_grouped,,,
0,13028,1063,7.543822
1,4410,445,9.165808
2+,2275,233,9.290271


**Вывод** Кол-во детей не сказывается негативно на выплатах, т.к. те у кого детей нет совсем отдают кредиты хуже

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [32]:
grouped_by_family_status['%'] = (grouped_by_family_status['sum'] / grouped_by_family_status['count']) * 100
display(grouped_by_family_status.sort_values('%'))

,count,sum,%
family_status,,,
вдовец / вдова,959,63,6.569343
в разводе,1195,85,7.112971
женат / замужем,12339,931,7.545182
гражданский брак,4151,388,9.347145
Не женат / не замужем,2810,274,9.750890


**Вывод** Самый низкий показатель у тех кто потерял мужу\жену. Можно предположить что потеря супруга или супруги сильно влияет на финансовое благополучие

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
income_pivot = df.pivot_table(index=['income_status'], columns=['debt'], values='education', aggfunc='count')
income_pivot['%'] = income_pivot[1] / (income_pivot[1] + income_pivot[0]) * 100
income_pivot

debt,0,1,%
income_status,,,
Высокий уровень дохода,6484,622,8.753166
Низкий уровень дохода,3086,262,7.825568
Сверхвысокий уровень дохода,5035,389,7.171829
Средний уровень дохода,5108,468,8.393113


**Вывод** Хуже всего платят люди со сверхвысокими доходами и низкими, если с низикими доходами еще логично предположить по какой причине они хуже платят, то со сверхвысокоми вариантов неуплаты по кредиту может быть на много больше.

- Как разные цели кредита влияют на его возврат в срок?

In [34]:
grouped_by_purpose = df.groupby('purpose_grouped')['debt'].agg(['count', 'sum'])
grouped_by_purpose['%'] = (grouped_by_purpose['sum'] / grouped_by_purpose['count']) * 100
display(grouped_by_purpose.sort_values('%'))

,count,sum,%
purpose_grouped,,,
недвижимость,10811,782,7.233373
свадьба,2324,186,8.003442
образование,4013,370,9.220035
автомобиль,4306,403,9.359034


**Вывод** Кредит на недвижимость и свадьбу наиболее проблемные, что в целом логично, кредит на недвижимость связаны с долгим сроком кредитования и не всегда удается поддерживать нужный уровень дохода на протяжение многих лет. Со свадьбой можно предположить что кредит на такие цели берут импульсивно в надежде погасить большую ее часть деньгами подаренными на свадьбу.

## Шаг 4. Общий вывод

Есть зависимости между целями кредита, семейным положением, уровнем дохода и количеством детей в семье. Нельзя сказать, что они кардинально отличаются, но 2% на больших числах могут оказать существенное влияние на кредитный портфель банка. Особое внимание следует уделить вдовам/вдовцам или молодоженам — с ними могут риски по невозврату кредитов выше